Cargar el dataset que contiene información sobre las películas y las puntuaciones de los usuarios.


In [1]:
import pandas as pd

In [ ]:
amazon='csv/amazon_prime_titles'
netflix ='csv/netflix_titles'
disney='csv/disney_plus_titles'
hulu='csv/hulu_titles'
archivos = [amazon,netflix,disney,hulu]
df = pd.read_csv(netflix+".csv")

lista_df = []
for archivo in archivos:
     df1 = pd.read_csv(archivo+".csv")
     lista_df.append(df1)

dfconbinado = pd.concat(lista_df)

In [47]:
# Obtener la ruta de la carpeta que contiene los archivos CSV
ac1='ratings/1'
ac2='ratings/2'
ac3='ratings/3'
ac4='ratings/4'
ac5='ratings/5'
ac6='ratings/6'
ac7='ratings/7'
archivos = [ac1,ac2,ac3,ac4,ac5,ac6,ac7]
lista_df = []
for archivo in archivos:
            df1 = pd.read_csv(archivo+".csv")
            lista_df.append(df1)

df_rating = pd.concat(lista_df)
df_rating = df_rating.rename(columns={'movieId': 'id'})
df_rating = df_rating.rename(columns={'rating': 'score'})

In [ ]:
df_rating.head(5)

In [ ]:
df_rating.head(5)

In [107]:
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import StandardScaler
import pandas as pd

df = df_rating
# Eliminar columnas innecesarias
df_merged = pd.merge(dfconbinado, df_rating, on='id')

# Eliminar cualquier columna no deseada
df_merged = df_merged[['id', 'score', 'title']]

#df = df.drop(['show_id','type','director','cast','country','date_added','release_year','rating','duration','listed_in','description','duration_int','duration_type'], axis=1)

print(df_merged)



            id  score            title
0         as14    2.5       resilencia
1         as14    3.0       resilencia
2         as14    4.0       resilencia
3         as14    2.0       resilencia
4         as14    3.0       resilencia
...        ...    ...              ...
221214  as9620    4.5  the shadow side
221215  as9620    4.0  the shadow side
221216  as9620    3.0  the shadow side
221217  as9620    5.0  the shadow side
221218  as9620    4.5  the shadow side

[221219 rows x 3 columns]


Calcular la matriz de similitud entre las películas basándose en las puntuaciones de los usuarios. Para ello, se puede utilizar una técnica como el cosine similarity.

In [ ]:
# Cargar el dataset
#df_merged = pd.read_csv('peliculas.csv', usecols=['id_pelicula', 'score', 'titulo'])

# Calcular la matriz de similitud
matriz_pivot = df_merged.pivot_table(index='id', columns='title', values='score')
matriz_similitud = matriz_pivot.corr()
print(matriz_similitud)


In [ ]:
df=df_merged
print(df.head(5))
# Convertir el DataFrame en una matriz
matriz_puntuaciones = df.pivot_table(index='title', columns='id', values='score')



In [95]:
import numpy as np
# Normalizar los datos
scaler = StandardScaler()
matriz_puntuaciones_norm = scaler.fit_transform(matriz_puntuaciones)
#matriz_puntuaciones_norm = np.nan_to_num(matriz_puntuaciones_norm)

In [92]:
# Calcular la matriz de similitud
matriz_similitud = cosine_similarity(matriz_puntuaciones_norm)

Definir la función get_recommendation(titulo: str) que tomará como entrada el título de la película y devolverá una lista con las 5 películas más recomendadas.

In [ ]:
from fastapi import FastAPI

app = FastAPI()

@app.get('/recommendation/{titulo}')
def get_recommendation(titulo: str):
    # Obtener el índice de la película en la matriz
    indice_pelicula = matriz_puntuaciones.index.get_loc(titulo)

    # Obtener la fila correspondiente en la matriz de similitud
    fila_similitud = matriz_similitud[indice_pelicula]

    # Obtener los índices de las 5 películas más similares
    indices_similares = fila_similitud.argsort()[::-1][1:6]

    # Obtener los nombres de las películas más similares
    peliculas_similares = [matriz_puntuaciones.index[i] for i in indices_similares]

    return peliculas_similares

